In [ ]:
from autumn.tools.project import get_project, ParameterSet
from matplotlib import pyplot as plt
from autumn.tools.plots.utils import REF_DATE
import pandas as pd

In [ ]:
project = get_project("covid_19", "manila")

In [ ]:
# run baseline model
model_0 = project.run_baseline_model(project.param_set.baseline)
derived_df = model_0.get_derived_outputs_df()

In [ ]:
# run scenarios
start_times = [
    sc_params.to_dict()["time"]["start"] for sc_params in project.param_set.scenarios
]
sc_models = project.run_scenario_models(model_0, project.param_set.scenarios, start_times=start_times)

In [ ]:
derived_dfs = [m.get_derived_outputs_df() for m in sc_models]

In [ ]:
outputs = ["notifications", "hospital_occupancy", "icu_occupancy"]

In [ ]:
sc_colors = ["blue", "red","green", "orange", "purple", "cornflowerblue"]

max_y = {
    "notifications": 6000, "hospital_occupancy": 7000, "icu_occupancy": 1300,
}

scenario_lists = [ [1, 2, 3], [4, 5, 6]]
titles = ["95% of 20+ fully vaccinated by end of 2021", "95% of 20+ and 50% of 15-19 fully vaccinated by end of 2021"]
filename_suf = ["", "_young_vacc"]
scenario_names = [
    "no mobility increase",
    "25% mobility increase",
    "50% mobility increase"
] * 2


for i_s, scenario_list in enumerate(scenario_lists):
    
    for output in outputs:
        fig = plt.figure(figsize=(12, 8))
        plt.style.use("ggplot")
        axis = fig.add_subplot()
        axis = derived_df[output].plot(color="black", label="baseline")

        for sc_id in scenario_list:
            d = derived_dfs[sc_id - 1]
            
            if output in d.columns:
                d[output].plot(color=sc_colors[sc_id - 1], label=scenario_names[sc_id - 1])
        axis.vlines(x=pd.Timestamp("2022-01-01"), ymin=0, ymax=max_y[output], color="black", linestyle="dashed")
        

        plt.text(x=pd.Timestamp("2022-01-05"), y=max_y[output] * .9, s="schools reopen", ha="left", fontsize=12)

        axis.set_ylim((0, max_y[output]))

        axis.set_xlim((pd.Timestamp("2021-09-01"), pd.Timestamp("2022-06-01")))
        axis.set_ylabel(output.replace("_", " "), fontsize=15)
        
        plt.title(titles[i_s], fontsize=15)
        
        plt.legend(fontsize=12, facecolor="white", markerscale=3)
        
        filename = f"ncr_11oct21_{output}{filename_suf[i_s]}"
        plt.savefig(filename,dpi=300)
        